In [25]:
p10 = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
p8 = (6, 3, 7, 4, 8, 5, 10, 9)

In [26]:
def permute_by_seq(s,p):
    res = ""
    for i in p:
        res += s[i-1]
    return res

In [27]:
#12345
#2345 1
def left_shift(s,n):
    res = s[n:] + s[:n]
    return res

In [28]:
def key_generation(key):
    key_after_p10 = permute_by_seq(key,p10)
    
    key_left = key_after_p10[0:5]
    key_right = key_after_p10[5:10]
    
    key_left = left_shift(key_left,1)
    key_right = left_shift(key_right,1)
    
    #key for round 1
    k1 = permute_by_seq(key_left + key_right,p8)
    
    key_left = left_shift(key_left,2)
    key_right = left_shift(key_right,2)
    
    #key for round 2
    k2 = permute_by_seq(key_left + key_right,p8)
    
    return k1,k2

In [29]:
k1, k2 = key_generation("1100011110")
print("key1 :" + k1," key2: " + k2)

key1 :11101001  key2: 10100111


In [30]:
initial_p = (2, 6, 3, 1, 4, 8, 5, 7)
inv_initial_p = (4, 1, 3, 5, 7, 2, 8, 6)

exp_p = (4, 1, 2, 3, 2, 3, 4, 1)
p4 = (2, 4, 3, 1)

sb_0 = [
            ["01", "00", "11", "10"],
            ["11", "10", "01", "00"],
            ["00", "10", "01", "11"],
            ["11", "01", "11", "10"]
         ]

sb_1 = [
            ["00", "01", "10", "11"],
            ["10", "00", "01", "11"],
            ["11", "00", "01", "00"],
            ["10", "01", "00", "11"]
         ]

In [33]:
def calculate_xor(s1,s2):
    res = ""
    for i in range(0,len(s1)):
        if s1[i] == s2[i]:
            res += "0"
        else:
            res += "1"
    return res

In [34]:
#0123
# row -> 03
# col -> 12
def look_sbox(s,sb):
    row = int(s[0])*2 + int(s[3])
    col = int(s[1])*2 + int(s[2])
    return sb[row][col]

In [35]:
def compute(plain_text,k1,k2):
    pt = permute_by_seq(plain_text,initial_p)
    
    #round 1
    lt = pt[0:4]
    rt = pt[4:8]
    
    ert = permute_by_seq(rt,exp_p)
    xor_rt = calculate_xor(ert,k1)
    
    x_lt = xor_rt[0:4]
    x_rt = xor_rt[4:8]
    
    sb_t = look_sbox(x_lt,sb_0) + look_sbox(x_rt,sb_1)
    
    r1 = permute_by_seq(sb_t,p4)
    r1 = calculate_xor(lt,r1)
    
    #round 2
    lt = rt
    rt = r1
    
    ert = permute_by_seq(rt,exp_p)
    xor_rt = calculate_xor(ert,k2)
    
    x_lt = xor_rt[0:4]
    x_rt = xor_rt[4:8]
    
    sb_t = look_sbox(x_lt,sb_0) + look_sbox(x_rt,sb_1)
    
    r2 = permute_by_seq(sb_t,p4)
    r2 = calculate_xor(lt,r2)
    
    
    #final contatenanted text
    ft = r2 + rt
    
    #cipher
    ct = permute_by_seq(ft,inv_initial_p)
    
    return ct

In [37]:
cipher = compute("00101000",k1,k2)
print("Cipher " + cipher)

Cipher10001010


In [39]:
plain = compute(cipher,k2,k1)
print("Plain " + plain)

Plain 00101000
